# Traveling waves of Nagumo type

We consider the Nagumo equation :

\begin{equation} 
\partial_t u - D \, \partial^2_x u = k \, u^2 (1 - u)
\end{equation}

In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.palettes import Category10, Category20
from bokeh.models import PrintfTickFormatter

from ipywidgets import FloatProgress, IntProgress
from IPython.display import display

from mylib.nagumo_model import nagumo_model

import mylib.integration as integration

output_notebook(hide_banner=True)

## Integration with Radau5

In [ ]:
def plot_radau_fortran_solution():
    
    k = 1.
    d = 1.

    xmin = -70.
    xmax = 70.
    nx = 1001
    tini = 0.0
    tend = 50.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn = nm.fcn_radau
    fcn_exact = nm.fcn_exact
    
    x = np.linspace(xmin, xmax, nx)
    yini = fcn_exact(tini)
    
    fig_sol = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, title="Solution")
    fig_sol.x(x, yini, color="grey", legend = "sol at t=0")
    fig_sol.legend.click_policy="hide"

    fig_err = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, title="Error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.2e")
    fig_err.legend.click_policy="hide"
    
    progress_bar = FloatProgress(min=tini, max=tend, value=0, description='Progress:',)
    display(progress_bar)
    
    nt_plot = 5
    t_plot = np.linspace(tini, tend, nt_plot)
    for it, ti in enumerate(t_plot[:-1]):
        sol = integration.radau5(t_plot[it], t_plot[it+1], yini, fcn, njac=1, atol=1.e-12, rtol=1.e-12)
        fig_sol.x(x, sol.y, legend=f"sol at t={t_plot[it+1]}", color=Category10[10][it])
        yexa = fcn_exact(t_plot[it+1])
        fig_err.x(x, np.abs(sol.y-yexa), legend=f"err at t={t_plot[it+1]}", color=Category10[10][it])
        yini = sol.y
        progress_bar.value = t_plot[it+1]
        
    show(column(fig_sol, fig_err))

plot_radau_fortran_solution()

## Integration with strang method

In [ ]:
def plot_strang_solution():
    
    k = 1.
    d = 1.
    xmin = -70.
    xmax = 70.
    nx = 1001
    tini = 0.0
    tend = 50.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_exact = nm.fcn_exact
    fcn = nm.fcn_radau
    fcn_diff = nm.fcn_diff_rock
    fcn_reac = nm.fcn_reac_radau
    
    x = np.linspace(xmin, xmax, nx)
    yini = nm.fcn_exact(tini)
    yini_qexa = nm.fcn_exact(tini)

    fig_sol = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, title="Solution")
    fig_sol.x(x, yini, color="grey", legend = "sol at t=0")
    
    fig_err = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950, title="Error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.2e")
    
    progress_bar = FloatProgress(min=tini, max=tend, value=0, description='Progress:',)
    display(progress_bar)

    # number of points per plot interval
    nt_strang = 41
    nt_plot = 6
    t_plot = np.linspace(tini, tend, nt_plot)
    for it, ti in enumerate(t_plot[:-1]):
        #print(t_plot[it], t_plot[it+1])
        ysol = integration.strang(t_plot[it], t_plot[it+1], nt_strang, yini, fcn_diff, fcn_reac)
        sol_qexa = integration.radau5(t_plot[it], t_plot[it+1], yini_qexa, fcn, njac=1, rtol=1.e-12, atol=1.e-12)
        yexa = fcn_exact(t_plot[it+1])
        #print(np.linalg.norm(yexa))
        yini = ysol
        yini_qexa = sol_qexa.y
        fig_sol.x(x, ysol, legend=f"sol at t={t_plot[it+1]}", color=Category20[20][2*it])
        fig_err.x(x, np.abs(ysol-yexa), legend=f"err (computed with exact sol) at t={t_plot[it+1]}", 
                  color=Category20[20][2*it])
        fig_err.x(x, np.abs(ysol-sol_qexa.y), legend=f"err (computed with quasi-exact sol)  at t={t_plot[it+1]}", 
                  color=Category20[10][2*it+1])
        progress_bar.value = t_plot[it+1]

    fig_err.legend.click_policy="hide"    
    fig_err.legend.location = "top_left"
    show(column(fig_sol, fig_err))

plot_strang_solution()

* 为了看时间积分的误差，使用相同的离散，但是把tol调到很高，和比较低的tolerance做比较。

## Order of strang method

In [ ]:
def plot_strang_order():
    
    k = 1.
    d = 1.
    xmin = -70.
    xmax = 70.
    nx = 1001
    tini = 0.0
    tend = 30.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_exact = nm.fcn_exact
    fcn = nm.fcn_radau
    fcn_diff = nm.fcn_diff_rock
    fcn_reac = nm.fcn_reac_radau
    
    x = np.linspace(xmin, xmax, nx)
    yini = nm.fcn_exact(tini)
    yini_qexa = nm.fcn_exact(tini)
    
    sol_qexa = integration.radau5(tini, tend, yini, fcn, njac=1, rtol=1.e-12, atol=1.e-12)
    
    nt = np.array([17, 33, 65, 129, 257])
    dt = (tend-tini)/(nt-1)
    err = np.empty(nt.size)
    
    progress_bar = IntProgress(min=0, max=nt.size, value=0, description='Progress:',)
    display(progress_bar)

    for it, nti in enumerate(nt):
        ysol = integration.strang(tini, tend, nti, yini, fcn_diff, fcn_reac)
        err[it] = np.linalg.norm(ysol-sol_qexa.y)/(nx-1)
        progress_bar.value = it+1
        
    fig_err = figure(plot_height=400, plot_width=950, x_axis_type="log", y_axis_type="log", title = "L2 norm of error")
    fig_err.x(dt, err, size=10)
    fig_err.line(dt, (err[0]/(dt[0]*dt[0]))*(dt*dt), color = "Red", legend="slope=2")
    fig_err.legend.location = "top_left"
    show(fig_err)
    
plot_strang_order()

## Wave speed and wave profile

## Radau integration (scipy version)

In [2]:
def compute_wave_speed():
    
    k = 1.
    d = 1.
    xmin = -70.
    xmax = 70.
    nx = 2001
    tini = 0.0
    tend = 50.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn_exact = nm.fcn_exact
    fcn = nm.fcn_radau
    fcn_diff = nm.fcn_diff_rock
    fcn_reac = nm.fcn_reac_radau
    
    x = np.linspace(xmin, xmax, nx)
    dx = (xmax-xmin)/(nx-1)

    yini = fcn_exact(tini)
    yini_qexa = fcn_exact(tini)
    
    # find x05 such as yini(x05) = 0.5
    for ix, yi in enumerate(yini):
        if (yi<0.5):
            alpha = (0.5 - yini[ix]) / (yini[ix-1] - yini[ix])
            x05_ini = x[ix] + (1-alpha)*(x[ix]-x[ix-1])
            break
 
    progress_bar = FloatProgress(min=tini, max=tend, value=0, description='Progress:',)
    display(progress_bar)

    # number of points per plot interval
    nt_strang = 51
    nt_plot = 5
    dt = ((tend-tini)/(nt_plot-1))/(nt_strang-1)

    v = np.zeros(nt_plot-1)
    v_exa = np.zeros(nt_plot-1)
    
    t_plot = np.linspace(tini, tend, nt_plot)

    for it, ti in enumerate(t_plot[:-1]):
        #print("dt_plot : ", t_plot[it], t_plot[it+1])
 
        ysol = integration.strang(t_plot[it], t_plot[it+1], nt_strang, yini, fcn_diff, fcn_reac)
        # find x05 such as ysol[x05] = 0.5
        for ix, yi in enumerate(ysol):
            if (yi<0.5):
                #print(ix, ysol[ix-1], yi)
                alpha = (0.5 - ysol[ix]) / (ysol[ix-1] - ysol[ix])
                x05 = x[ix] + (1-alpha)*(x[ix]-x[ix-1])
                break
        # compute wave speed between t_plot[it] and t_plot[it+1]
        v[it] = (x05 - x05_ini) / (t_plot[it+1] - t_plot[it])

        sol_qexa = integration.radau5(t_plot[it], t_plot[it+1], yini_qexa, fcn, njac=1, rtol=1.e-12, atol=1.e-12)
        # find x05 such as sol_qexa.y[x05] = 0.5
        for ix, yi in enumerate(sol_qexa.y):
            if (yi<0.5):
                #print(ix, ysol[ix-1], yi)
                alpha = (0.5 - sol_qexa.y[ix]) / (sol_qexa.y[ix-1] - sol_qexa.y[ix])
                x05 = x[ix] + (1-alpha)*(x[ix]-x[ix-1])
                break
        # compute wave speed between t_plot[it] and t_plot[it+1]
        v_exa[it] = (x05 - x05_ini) / (t_plot[it+1] - t_plot[it])
        
        yini = ysol
        yini_qexa = sol_qexa.y
        x05_ini = x05
        progress_bar.value = t_plot[it+1]
        
    print("Wave speed (computed with solution obtained with strang method): ", np.mean(v))
    print("Wave speed (computed with quasi-exact solution)                : ", np.mean(v_exa))
    
    dyoverdx = np.zeros(nx)
    for ix in range(1,nx-1):
        dyoverdx[ix] = (ysol[ix+1] - ysol[ix-1]) / (2*dx)
          
    dyoverdx_exa = (-1/np.sqrt(2))*ysol*(1-ysol)
    
    print(dyoverdx_exa.size, dyoverdx.size)
        
    fig_phase = figure(plot_height=400, plot_width=900, title="Phase space : strang formula dt = " + ("%.4f" % dt))
    fig_phase.x(ysol, dyoverdx)
    fig_err = figure(plot_height=400, plot_width=900, title = "Error between numerical solution and exact solution")
    fig_err.x(ysol, np.abs(dyoverdx-dyoverdx_exa))

    show(column(fig_phase, fig_err))

compute_wave_speed()

FloatProgress(value=0.0, description='Progress:', max=50.0)

OSError: [WinError 126] 找不到指定的模块。

* 为什么误差随时间更大了？
* 两个波shifting？比较接近？velocity？
* 提高stiffness有可能order不再是2阶方法了，strang 可能掉到1阶或者1.5